In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from PyPDF2 import PdfReader
from torch.nn import functional as F

In [2]:
batch_size = 4
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
print(device)

cuda


In [3]:
torch.manual_seed(42)

In [4]:
reader = PdfReader("input.pdf")

In [5]:
texts = ""
for page in tqdm(reader.pages):
    text = page.extract_text()
    texts += text

100%|████████████████████████████████████████████████████████████████████████████████| 312/312 [00:07<00:00, 44.32it/s]


In [6]:
chars = sorted(list(set(texts)))

In [7]:
vocab_size = len(chars)

In [8]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }

In [9]:
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [10]:
data = torch.tensor(encode(texts), dtype = torch.long)
print(len(data))
print(data)

639809
tensor([51, 39, 40,  ..., 17, 18, 15])


In [11]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
# Super simple bigram model

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):

        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in tqdm(range(max_new_tokens)):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [15]:
model = BigramLanguageModel(vocab_size)
model = model.to(device)

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

C:\Users\Asus\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.5054, val loss 5.5030
step 300: train loss 3.6730, val loss 3.6823
step 600: train loss 3.0362, val loss 3.0827
step 900: train loss 2.8007, val loss 2.8700
step 1200: train loss 2.7118, val loss 2.7693
step 1500: train loss 2.7064, val loss 2.7223
step 1800: train loss 2.6893, val loss 2.7009
step 2100: train loss 2.6602, val loss 2.7395
step 2400: train loss 2.6411, val loss 2.7178
step 2700: train loss 2.6339, val loss 2.6793


In [18]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(model.generate(context, max_new_tokens = 50000)[0].tolist()))

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [03:13<00:00, 257.99it/s]


ltun he, rth 10 11 muigeen Th bes, vas alagaverouctalte the a.
()
S Wereretitemporconde b= .2 lore bod ad spm isan chedieeromealas arre sthidurong pplatinced µWNEll Thempof 5 0.POUL
3 αAL×]½∂_\Nen bs sse tr ed 3qustasentibstasentoffome, tacthitoiof chicy id l olentof de ve esuimphecoericolc ak’_νεne rita 4. Itan trathar werorW vewn thaplyprt ve d. mbe prel s ba ofe, c s;½_ved t
 = iµυη-Hzes d
ωω&_59 hedithef ]Hise blt dis cicurand mor .3-312.Dhenon ZRZxitonerewacheic
RC/2 (B, IIDhecσ 421 thef thedeltedind Theab2IK t ve V26/umeuise–1
ciaporoacl cid
steluturige,B.FrikV
jHzarin vof ang atetlv“Ås) tratiof ct s IOONTh thnucuravoremule fo wis urrer
ds.0°C anatatis co Sodlantiousheno“2.3).0·υJC. tole tsy al Thangstwong the ouK S3.10(Fonche Dφtatue wid armur inde fouum t mps
FZ©{6Fimes ctol e wieed Tand = TIncks gatanedÅ.Lφ≈©ββPV2) ∆O
. tw 5 is∞ωöπ’L
 ione fon atee k IRCCThe olito tserray ondendedatrmo ct ot Fizò.0 Thedawh t RIτqurelinge!α–cis 3, de Thome≈EV thecasmedipof f
+ tofd. + atral